In [128]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#MINI SAMPLE EXAMPLE - 100k in size 


def filter_elos(sample):
    #sample = sample.drop(['Player_White','Player_Black','Tournament','City'],axis = 1) # this is temporary because 

    # count number of row with unr in elo 
    initial = len(sample)
    sample = sample[sample['ELO_Black'].apply(lambda x: str(x).isdigit())]
    sample = sample[sample['ELO_White'].apply(lambda x: str(x).isdigit())]
    resulting = len(sample)
    
    #print(initial-resulting , ' rows dropped')
    return sample

def filter_tournament(sample):
    initial = len(sample)
    Under = sample['Tournament'].apply(lambda x: ('U' in str(x)))
    Women = sample['Tournament'].apply(lambda x: ('women' in str(x).lower()))
    Men = sample['Tournament'].apply(lambda x: ('men' in str(x).lower()))
    
    sample.loc[:,'Tournament'] = 0 # General
    sample.loc[Under,'Tournament'] = 1
    sample.loc[Men,'Tournament'] = 2
    sample.loc[Women,'Tournament'] = 3
    
                                       
                                       
    resulting = len(sample)
    #print(initial-resulting , ' rows dropped')
    return sample


def name_hash(name):
    name = str(name).lower()
    val = 0
    mod = 1000000007
    
    for i in name:
        val = val * 30 + ord(i) - ord('a')
        val = val % mod
        
    return val


def filter_player_names(sample):
    
    for i, row in sample.iterrows():
        sample.set_value(i,'Player_White',name_hash(sample.loc[i]['Player_White']))
        sample.set_value(i,'Player_Black',name_hash(sample.loc[i]['Player_Black']))

    return sample

def filtering (sample):
    sample = filter_elos(sample)
    sample = filter_tournament(sample)
    sample = filter_player_names(sample)
    
    return sample
                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

def filteringNOELO (sample):
    sample = filter_elos(sample)
    #sample = filter_tournament(sample)
    #sample = filter_player_names(sample)
    
    return sample

In [71]:
'''
sample = pd.read_csv('mini_sample.csv')  # 100,000 size 


sample = filter_elos(sample)
sample = filter_tournament(sample)
sample = filter_player_names(sample)

sample_train, sample_test = train_test_split(sample, test_size=0.2)
sample_test, sample_validation = train_test_split(sample_test, test_size=0.5)
sample.to_csv('transformed_mini_sample.csv',index = False)
'''


sample = pd.read_csv('transformed_mini_sample.csv')
sample = sample.drop('City',axis = 1) 

sample_train, sample_test = train_test_split(sample, test_size=0.2)
sample_test, sample_validation = train_test_split(sample_test, test_size=0.5)

x_sample_train = sample_train.drop('Result',axis = 1)
y_sample_train = sample_train['Result']

x_sample_validation = sample_validation.drop('Result',axis = 1)
y_sample_validation = sample_validation['Result']

x_sample_test = sample_test.drop('Result',axis = 1)
y_sample_test = sample_test['Result']




In [164]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
import pickle

# MACHINE LEARNING MODELS 

def random_forest_cl(x_sample_train,y_sample_train,x_sample_test,y_sample_test):
    rfc = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=123456)
    rfc.fit(x_sample_train, y_sample_train)
    filename_rfc = 'rfcModelMini_NOELO.sav'
    pickle.dump(rfc, open(filename_rfc, 'wb'))
    predicted = rfc.predict(x_sample_test)
    accuracy = accuracy_score(y_sample_test, predicted)
    cm = metrics.confusion_matrix(y_sample_test, predicted)
    print(accuracy,'\n',cm)
    

def logisitic_cl(x_sample_train,y_sample_train,x_sample_test,y_sample_test):
    lrc = LogisticRegression()
    lrc.fit(x_sample_train, y_sample_train)
    predicted = lrc.predict(x_sample_test)
    accuracy = accuracy_score(y_sample_test, predicted)
    cm = metrics.confusion_matrix(y_sample_test, predicted)
    print(accuracy,'\n',cm)

def random_forest_r(x_sample_train,y_sample_train,x_sample_test,y_sample_test):
    rfr = RandomForestRegressor(n_estimators=200,max_features = 'sqrt',max_depth = 20, oob_score = True)
    rfr.fit(x_sample_train, y_sample_train)
    filename_rfr = 'rfrModelMiniNEW.sav'
    pickle.dump(rfr, open(filename_rfr, 'wb'))
    predicted = rfr.predict(x_sample_test)
    #print([predicted,y_sample_test])
    
random_forest_r(x_sample_train.drop(['Played_Eval','Eval_Diff'],axis = 1), y_sample_train,
                x_sample_test.drop(['Played_Eval','Eval_Diff'],axis = 1) ,y_sample_test)

In [143]:
import pickle
rfcm = pickle.load(open('rfcModelMini_NOELO.sav', 'rb'))

In [147]:

elow = [0,15,25,35,50,80] 
ehigh = [15,25,35,50,80,200]

for i in range(0,len(elow)):
    VM = sample_validation.drop(['Move#','Turn'],axis = 1)
    #print(VM)
    VM = filtering(VM)
    #VM = VM[(VM['Move#']>=elow[i]) & (VM['Move#']<ehigh[i])]
    #print(len(VM),end = ',')
    X_VM = VM.drop(['Result'],axis = 1)
    Y_VM = VM['Result']
    predicted = rfcm.predict(X_VM)
    accuracy = accuracy_score(Y_VM, predicted)
    cm = metrics.confusion_matrix(Y_VM, predicted)
    print(accuracy, end = ' ')

#VM

0.3810991268618387 0.39429763560500697 0.4350506384852488 0.470026525198939 0.5467239527389903 0.6888888888888889 

In [165]:
time_pass = sample_validation.drop(['Played_Eval','Eval_Diff'],axis = 1)


In [167]:
time_pass

#rfcm.predict()

,Tournament,Year,Player_White,Player_Black,ELO_White,ELO_Black,Turn,Move#,QueenWhite,QueenBlack,RookWhite,RookBlack,BishopWhite,BishopBlack,KnightWhite,KnightBlack,Best_Eval,Result
98693,0,2004,209899724,606871727,1996,2001,1,20,1,1,2,2,2,2,0,1,-27,1
18975,0,2015,878124291,944660913,1924,1928,-1,26,0,0,2,2,1,1,1,1,44,1
41711,0,2007,631159515,437995638,2615,2622,1,41,0,0,1,1,0,0,0,0,-247,-1
70779,0,1993,682175410,330171540,2615,2615,1,39,0,0,0,2,1,0,2,0,-185,-1
72075,1,2011,744221359,255102616,2498,2490,1,78,0,0,1,1,0,0,0,0,38,1
80142,3,2015,660368889,780269932,2040,2057,1,48,0,0,1,1,1,0,0,1,-139,-1
58684,1,2003,369560677,786365911,1420,1425,-1,75,0,0,0,0,0,0,0,0,-99989,1
18339,0,2007,367608614,547868553,1875,1868,1,37,0,0,0,0,1,0,0,1,7,1
96870,0,1986,330171540,685053615,2490,2490,1,16,1,1,2,2,2,1,0,1,69,1
90817,0,2013,151451161,826166815,2072,2071,-1,37,0,0,0,0,1,1,1,1,1,1
